In [ ]:
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
#from keras.utils.data_utils import get_file
import numpy as np

import random
import sys
import io
import re

In [ ]:
# path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

path = 'data2.txt'
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()

text = re.sub(r'<.*>', '', text)
text = re.sub(r'\n', ' ', text)
text = re.sub(r' +', ' ', text)
text = re.sub(r'[0-9]', ' ', text)

print('corpus length:', len(text))

corpus length: 59842


In [ ]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 1946


In [ ]:
maxlen = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

nb sequences: 19934
Vectorization...


<ipython-input-4-7de617b7b4c4>:12: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
<ipython-input-4-7de617b7b4c4>:13: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(chars)), dtype=np.bool)


In [ ]:
print('Build model...')
model = Sequential()
model.add(LSTM(1024, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.001))

Build model...


In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    print('\n----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
#     for diversity in [0.2, 0.5, 1.0, 1.2]:
#         print('----- diversity:', diversity)

    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, 0.5)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [ ]:
model.fit(x, y, batch_size=128, epochs=60, callbacks=[print_callback])

Epoch 1/60
156/156 [==============================] - ETA: 0s - loss: 5.0284
----- Generating text after Epoch: 0
----- Generating with seed: "난디 그 뒤를 바라오니 그 때여 이 도령 비룡같은 노새등 뚜렷이 올라앉어"
난디 그 뒤를 바라오니 그 때여 이 도령 비룡같은 노새등 뚜렷이 올라앉어      "          도        루지         曹    만        "       (              니                                                                                    천    님 이   나 향                 도                              (소    내    )지  자              에     나   어               을     지        니           이 이            (              고     (니  (  이  은    하   니       이         을    느    시            
156/156 [==============================] - 43s 266ms/step - loss: 5.0284
Epoch 2/60
156/156 [==============================] - ETA: 0s - loss: 4.8395
----- Generating text after Epoch: 1
----- Generating with seed: " 상하야 아무 대답을 아니허고 대문안을 들어스니 그 때여 춘향 어머니난 "
 상하야 아무 대답을 아니허고 대문안을 들어스니 그 때여 춘향 어머니난 가            어       가 이      이  도     드라                리         이리    아  